Відкриття гугл диска

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!cp '/content/drive/MyDrive/images.zip' '/content/'
!unzip /content/images.zip -d /content/

In [ ]:
!pip install imgaug
!pip install numpy==1.24.4

In [ ]:
import os
import os.path
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import random
from torchvision import transforms
import math
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.amp import autocast, GradScaler
import torchvision.transforms.functional as TF
# import imgaug.augmenters as iaa

Дата лоадер

In [ ]:
def get_pathes(base_folder ):
    paths = []
    for i in range(11):
        folder_name = f'{i:02d}'
        image_folder = os.path.join(base_folder, folder_name, 'image_0')
        movements_file = os.path.join(base_folder, folder_name, f'{folder_name}.txt')
        paths.append((image_folder,movements_file))
    return paths

def get_data(batch_size, base_folder = '/content/images'):
    paths = get_pathes(base_folder)

    # Вибір папок для тренування, валідації і тестування
    validation_folder = paths[-2]  # Предостання папка
    test_folder = paths[-1]        # Остання папка
    train_folders = paths[:-2]     # Всі інші папки

    # Читання рухання з файлу
    def create_movements(movements_file):
        movements = []
        with open(movements_file, 'r') as f:
            for line in f:
                data = list(map(float, line.strip().split()))
                movements.append(data)
        return movements

    step = 1

    def transform_mov(movements):
        res_movements = []
        # for i in range(len(movements)-1):
        for i in range(len(movements)-step):
            m1 = torch.tensor(movements[i])
            m2 = torch.tensor(movements[i+step])
            res = m2-m1
            distance = math.sqrt(pow(res[3],2) + pow(res[7],2) + pow(res[11],2))
            res_movements.append([distance ])
        return res_movements

     # класс Dataset
    class ImageDataset(Dataset):
        def __init__(self, image_folder, movements, transform=None ):
            # Отримуємо список всіх файлів .png і сортуємо їх
            all_image_files = sorted([os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')])
            self.image_files = all_image_files
            self.movements = movements
            self.transform = transform

        def __len__(self):
            # Повертаємо мінімальну довжину між рухами та зображеннями - 1
            return min(len(self.movements), len(self.image_files) - 5)


        def __getitem__(self, idx):
            pathes = self.image_files[idx: idx + 5*step :step]
            images = [Image.open(p).convert('L') for p in pathes]


            # if self.transform:
            #     image1, image2 = self.transform(image1, image2)
            #     image1 = transforms.ToTensor()(image1)
            #     image2 = transforms.ToTensor()(image2)
            if self.transform:
                images = [self.transform(img) for img in images]

            # if self.transform:
            #     image1,image2 = self.transform(image1,image2)
            images = torch.stack(images, dim=0)  # [5, 1, H, W]
            images = images.permute(1, 0, 2, 3)  # [1, 5, H, W]
            movement = self.movements[idx + 4*step].clone().float() if isinstance(self.movements[idx+ 4*step], torch.Tensor) else torch.tensor(self.movements[idx + 4*step], dtype=torch.float32)
            return images, movement


    # Трансформації для зменшення використання пам'яті
    transform = transforms.Compose([
        transforms.Grayscale(),
        # transforms.Resize((376,1240)),
        # transforms.CenterCrop((370,1226)),
        transforms.Resize((180,320)),
        transforms.ToTensor()
    ])

    # transform = PairTransform()

    def load_dataset(folders):
        images, movements = [], []
        for image_folder, movements_file in folders:
            mov = create_movements(movements_file)
            mov = transform_mov(mov)
            movements.append(mov)
            images.append(image_folder)
        return images, movements

    train_images, train_movements = load_dataset(train_folders)
    train_dataset = []
    for index,image_folder in enumerate(train_images):
        dataset = ImageDataset(image_folder, train_movements[index], transform=transform)
        train_dataset.append(dataset)
    train_dataset = torch.utils.data.ConcatDataset(train_dataset)

    # Завантаження валідаційних даних
    val_images, val_movements = load_dataset([validation_folder])
    val_dataset = ImageDataset(val_images[0], val_movements[0], transform=transform)


   # Завантаження тестових даних
    test_images, test_movements = load_dataset([test_folder])
    test_dataset = ImageDataset(test_images[0], test_movements[0], transform=transform)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True, num_workers=4, pin_memory=True)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False, num_workers=4, pin_memory=True)

    return train_loader, validation_loader, test_loader


EVALUATION


In [ ]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def compute_accuracy(model, data_loader, device):
    model.eval()
    total_loss = 0.0
    num_batches = 0
    correct_predictions = 0   # Лічильник коректних передбачень
    total_samples = 0         # Загальна кількість елементів

    criterion = torch.nn.MSELoss()

    with torch.no_grad():
        for batch_idx, ((feature1, feature2),targets) in enumerate(data_loader):
            feature1 = feature1.to(device)
            feature2 = feature2.to(device)
            targets = targets.to(device)
            with autocast(device_type='cuda'):
                predicted_y = model(feature1, feature2)

            # Обчислення помилки
            loss = criterion(predicted_y, targets)
            total_loss += loss.item()
            num_batches += 1

            # Перевірка критерію якості
            relative_difference = torch.abs((predicted_y - targets)) # Розрахунок абсолютного відхилення
            valid_predictions = torch.all(relative_difference <= 0.1, dim=1)    # Умова: відхилення <= 10 см
            correct_predictions += torch.sum(valid_predictions).item()
            total_samples = total_samples + targets.size(0)


    avg_loss = total_loss / num_batches             # Розрахунок середньої помилки
    accuracy = correct_predictions / total_samples  # Розрахунок частки вірних передбачень
    return avg_loss, accuracy

def compute_accuracy1(model, data_loader, device):
    model.eval()
    total_loss = 0.0
    num_batches = 0
    correct_predictions = 0   # Лічильник коректних передбачень
    total_samples = 0         # Загальна кількість елементів

    criterion = torch.nn.MSELoss()

    with torch.no_grad():
        for batch_idx, (images,targets) in enumerate(data_loader):
            images = images.to(device)
            targets = targets.to(device)
            with autocast(device_type='cuda'):
                predicted_y = model(images)

            # Обчислення помилки
            loss = criterion(predicted_y, targets)
            total_loss += loss.item()
            num_batches += 1

            # Перевірка критерію якості
            relative_difference = torch.abs((predicted_y - targets)) # Розрахунок абсолютного відхилення
            valid_predictions = torch.all(relative_difference <= 0.1, dim=1)    # Умова: відхилення <= 10 см
            correct_predictions += torch.sum(valid_predictions).item()
            total_samples = total_samples + targets.size(0)


    avg_loss = total_loss / num_batches             # Розрахунок середньої помилки
    accuracy = correct_predictions / total_samples  # Розрахунок частки вірних передбачень
    return avg_loss, accuracy


Model

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        # Завантажуємо переднавчену модель ResNet50
        weights = ResNet50_Weights.DEFAULT
        base_model = resnet50(weights=weights)

        # Міняжмо перший conv слой для 1-канального зображенняЗ
        self.backbone = nn.Sequential()
        conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Копіюємо та усереднюємо ваги по каналам
        with torch.no_grad():
            conv1.weight[:] = base_model.conv1.weight.mean(dim=1, keepdim=True)
        self.backbone.add_module("conv1", conv1)

        #Додаємо інші блоки ResNet до avgpool (виключая fc)
        for name, module in list(base_model.named_children())[1:-2]:
            self.backbone.add_module(name, module)

        self.pool = nn.AdaptiveAvgPool2d((1, 3))
        self.lstm = nn.LSTM(input_size=2048 * 3, hidden_size=128, num_layers=1, batch_first=True)

        self.classifier = nn.Linear(128, 1)


    def forward(self,images):# images: [B, 5, 1, H, W]
        B,T,C,H,W = images.shape
        pairs = []
        for t in range(1, T):
            pair = torch.cat((images[:, t-1], images[:, t]), dim=1)  # [B, 2, H, W]
            features = self.backbone(pair)  # [B, 2048, H', W']
            features = self.pool(features)  # [B, 2048, 1, 3]
            features = features.view(B, -1)  # [B, 2048 * 3]
            pairs.append(features)

        seq = torch.stack(pairs, dim=1)  # [B, 4, 2048*3]

        lstm_out, _ = self.lstm(seq)  # [B, 4, 128]
        last_output = lstm_out[:, -1, :]  # [B, 128]
        out = self.classifier(last_output)  # [B, 1]
        return out

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        # Завантажуємо переднавчену модель ResNet50
        weights = ResNet50_Weights.DEFAULT
        base_model = resnet50(weights=weights)

        # Міняжмо перший conv слой для 1-канального зображенняЗ
        self.backbone = nn.Sequential()
        conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Копіюємо та усереднюємо ваги по каналам
        with torch.no_grad():
            conv1.weight[:] = base_model.conv1.weight.mean(dim=1, keepdim=True)
        self.backbone.add_module("conv1", conv1)

        #Додаємо інші блоки ResNet до avgpool (виключая fc)
        for name, module in list(base_model.named_children())[1:-2]:
            self.backbone.add_module(name, module)

        self.lstm1 = nn.LSTMCell(input_size = 2048*1*3, hidden_size=128)
        self.lstm2 = nn.LSTMCell(input_size = 128, hidden_size=128)

        self.classifier = nn.Linear(128, 1)


    def forward(self,image1,image2):
          images = torch.cat((image1,image2),dim = 1)

          out_features = self.backbone(images)
          out_features = torch.nn.functional.adaptive_avg_pool2d(out_features, (1,3 ))  # [B, C, 3, 10]
          out_features = out_features.view(out_features.size(0), -1)  # [B, C * 3 * 10]

        #   out_features = torch.flatten(out_features, 1)
          batch_size = out_features.size(0)

          # Ініціалізуємо приховані стани та стани осередків для lstm1
          h_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features.device)
          c_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features.device)

          # Пропускаємо ознаки першої картинки через lstm1 (один часовий крок)
          h_t1_next, c_t1_next = self.lstm1(out_features, (h_t1, c_t1))

        # Пропускаємо ознаки першої картинки через lstm2 (один часовий крок)
          h_t2_next, c_t2_next = self.lstm2(h_t1_next, (h_t1_next, c_t1_next))

          out = self.classifier(h_t2_next) # [batch_size, 1]
          return out

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=(3, 5, 5), stride=(1, 2, 2), padding=(0, 2, 2))
        self.conv2 = nn.Conv3d(16, 32, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
        self.conv3 = nn.Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
        with torch.no_grad():
            dummy = torch.zeros(1, 1, 5, 180, 320)
            out = self._forward_conv(dummy)
            self.flatten_size = out.view(1, -1).shape[1]
        # Output size after convs: (B, 64, 1, 46, 153)
        self.fc1 = nn.Linear(self.flatten_size, 512)
        self.fc2 = nn.Linear(512, 1)  # [x, y, z, roll, pitch, yaw]

    def _forward_conv(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

Train


In [ ]:
def train(model, num_epochs, last_epoch, train_loader,
                validation_loader, test_loader,optimizer, device,scheduler,
          checkpoint_dict,scaler):
    start_time = time.time()
    minibatch_loss_list, train_loss_list, valid_loss_list = [] ,[], []
    criterion = torch.nn.MSELoss()

    for epoch in range(last_epoch, num_epochs):
        model.train()
        for batch_idx,(images,targets) in enumerate(train_loader):
            # feature1 = feature1.to(device)
            # feature2 = feature2.to(device)
            images = images.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            # <<< Mixed precision begins
            with autocast(device_type='cuda'):
                predicted_y = model(images)
                loss = criterion(predicted_y, targets)

            #  Зворотній прохід
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            minibatch_loss_list.append(loss.item())
            if batch_idx %100 == 0:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_loader):04d} '
                      f'| Loss: {loss:.4f}'
                      f'| Learning rate: {optimizer.param_groups[0]["lr"]}')
        # scheduler.step()
        model.eval()
        with torch.no_grad():
            avg_loss_train, accuracy_train = compute_accuracy1(model, train_loader, device)
            avg_loss_val, accuracy_val = compute_accuracy1(model, validation_loader, device)
            # scheduler.step( avg_loss_val)
            print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                  f'| Train AVG LOSS: {avg_loss_train: .4f} | Accuracy_train: {accuracy_train: .4f} \n'
                  f'| Validation AVG LOSS: {avg_loss_val: .4f} | Accuracy_val: {accuracy_val: .4f} ')
            train_loss_list.append(avg_loss_train)
            valid_loss_list.append(avg_loss_val)

        scheduler.step()

        # checkpoint = {
        #     'model_state_dict': model.state_dict(),
        #     'optimizer_state_dict': optimizer.state_dict()
        # }

        checkpoint_dict['state_model'] = model.state_dict()
        checkpoint_dict['state_opt'] =  optimizer.state_dict()
        checkpoint_dict[ 'state_scheduler'] = scheduler.state_dict()
        checkpoint_dict['train_loss'] = avg_loss_train
        checkpoint_dict['val_loss'] = avg_loss_val
        checkpoint_dict['train_acc'] = accuracy_train
        checkpoint_dict['val_acc'] =  accuracy_val
        checkpoint_dict['EPOCHS'] = num_epochs
        checkpoint_dict['current_epoch'] = epoch + 1
        checkpoint_dict[ 'learning'] = optimizer.param_groups[0]["lr"]

        path = directory_path + str(epoch + 1) + '_epoch.pth'
        torch.save(checkpoint_dict, path)
        # path = '/content/drive/My Drive/checkpoint_model_3.pth'
        # torch.save(checkpoint, path)

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')

    elapsed = (time.time() - start_time)/60
    print(f'Total Training Time: {elapsed:.2f} min')

    avg_loss_test, accuracy_test = compute_accuracy1(model, test_loader,device)
    print(f'Test AVG LOSS: {avg_loss_test: .4f} | Accuracy_test: {accuracy_test: .4f}')

    return minibatch_loss_list, train_loss_list, valid_loss_list


Опис моделі, оптимізатора, шедулера

In [ ]:
txt_model_opt_shed = '''
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=(3, 5, 5), stride=(1, 2, 2), padding=(0, 2, 2))
        self.conv2 = nn.Conv3d(16, 32, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
        self.conv3 = nn.Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
        with torch.no_grad():
            dummy = torch.zeros(1, 1, 5, 180, 320)
            out = self._forward_conv(dummy)
            self.flatten_size = out.view(1, -1).shape[1]
        # Output size after convs: (B, 64, 1, 46, 153)
        self.fc1 = nn.Linear(self.flatten_size, 512)
        self.fc2 = nn.Linear(512, 1)  # [x, y, z, roll, pitch, yaw]

    def _forward_conv(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

'''

Створення бібліотеки для збеження збережень

In [ ]:
directory_path = '/content/drive/My Drive/checpoint/3dcnn/'

# Перевірка існування директорії
if not os.path.exists(directory_path):
    # Створення директорії
    os.makedirs(directory_path)
    print(f"Директорія {directory_path} була створенна.")
else:
    print(f"Директорія {directory_path} вже існує.")

Директорія /content/drive/My Drive/checpoint/3dcnn/ була створенна.


Створення словника для збереження результатів

In [ ]:
checkpoint_dict = {
    'model_description' : txt_model_opt_shed,
    'state_model' : None,
    'state_opt'   : None,
    'state_scheduler' : None,

    'train_loss' :None,
    'val_loss'   : None,
    'best_loss'  : None,
    'train_acc'  : None,
    'val_acc'    : None,

    'EPOCHS'     : None,
    'current_epoch' : None,
    'learning' : None
}

MAIN

In [ ]:
BATCH_SIZE = 8
RANDOM_SEED = 123
NUM_EPOCHS = 5
last_epoch = 0
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Використовується пристрій: {DEVICE}")

Використовується пристрій: cuda


In [ ]:
from torch.optim.lr_scheduler import StepLR
set_all_seeds(RANDOM_SEED)

model = Network()
model = model.to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.0006)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
train_loader, validation_loader, test_loader = get_data(batch_size = BATCH_SIZE)
# scheduler = StepLR(optimizer, step_size = 4, gamma = 0.3)
scheduler = None
scaler = GradScaler()
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 90,465,057 trainable parameters


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Завантаження словника з файлу
path = './3_epoch.pth'
checkpoint = torch.load(path)
if scheduler == None:
    scheduler = StepLR(optimizer, step_size = 1, gamma = 1)
# # Завантаження стану моделі
# save_check = checkpoint['optimizer_state_dict']
# save_check['lr'] = 0.0003
model.load_state_dict( checkpoint['state_model'] )
optimizer.load_state_dict( checkpoint['state_opt'] )
scheduler.load_state_dict( checkpoint['state_scheduler'] )
last_epoch = checkpoint['current_epoch']
# # Завантаження стану оптимізатора
# optimizer.load_state_dict(save_check)

In [ ]:
avg_loss_test, accuracy_test = compute_accuracy1(model, test_loader,DEVICE)
print(f'Test AVG LOSS: {avg_loss_test: .4f} | Accuracy_test: {accuracy_test: .4f}')

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Test AVG LOSS:  0.0524 | Accuracy_test:  0.4565


In [ ]:
if scheduler == None:
    scheduler = StepLR(optimizer, step_size = 1, gamma = 1)
minibatch_loss, train_acc, valid_acc = train(model = model,
            num_epochs = NUM_EPOCHS,
            last_epoch = last_epoch,
            train_loader = train_loader,
            validation_loader = validation_loader,
            test_loader = test_loader,
            optimizer = optimizer,
            device = DEVICE,
            scheduler = scheduler,
            checkpoint_dict = checkpoint_dict,
            scaler = scaler)

Epoch: 001/005 | Batch 0000/2546 | Loss: 1.0446| Learning rate: 0.0001
Epoch: 001/005 | Batch 0100/2546 | Loss: 0.2923| Learning rate: 0.0001
Epoch: 001/005 | Batch 0200/2546 | Loss: 0.1982| Learning rate: 0.0001
Epoch: 001/005 | Batch 0300/2546 | Loss: 0.1440| Learning rate: 0.0001
Epoch: 001/005 | Batch 0400/2546 | Loss: 0.0769| Learning rate: 0.0001
Epoch: 001/005 | Batch 0500/2546 | Loss: 0.0596| Learning rate: 0.0001
Epoch: 001/005 | Batch 0600/2546 | Loss: 0.0659| Learning rate: 0.0001
Epoch: 001/005 | Batch 0700/2546 | Loss: 0.0325| Learning rate: 0.0001
Epoch: 001/005 | Batch 0800/2546 | Loss: 0.2019| Learning rate: 0.0001
Epoch: 001/005 | Batch 0900/2546 | Loss: 0.0363| Learning rate: 0.0001
Epoch: 001/005 | Batch 1000/2546 | Loss: 0.0186| Learning rate: 0.0001
Epoch: 001/005 | Batch 1100/2546 | Loss: 0.0248| Learning rate: 0.0001
Epoch: 001/005 | Batch 1200/2546 | Loss: 0.0652| Learning rate: 0.0001
Epoch: 001/005 | Batch 1300/2546 | Loss: 0.0131| Learning rate: 0.0001
Epoch:

In [ ]:
if scheduler == None:
    scheduler = StepLR(optimizer, step_size = 1, gamma = 1)
minibatch_loss, train_acc, valid_acc = train(model = model,
            num_epochs = NUM_EPOCHS,
            last_epoch = 3,
            train_loader = train_loader,
            validation_loader = validation_loader,
            test_loader = test_loader,
            optimizer = optimizer,
            device = DEVICE,
            scheduler = scheduler,
            checkpoint_dict = checkpoint_dict,
            scaler = scaler)

Epoch: 004/005 | Batch 0000/2546 | Loss: 0.0029| Learning rate: 0.0001
Epoch: 004/005 | Batch 0100/2546 | Loss: 0.0031| Learning rate: 0.0001
Epoch: 004/005 | Batch 0200/2546 | Loss: 0.0031| Learning rate: 0.0001
Epoch: 004/005 | Batch 0300/2546 | Loss: 0.0015| Learning rate: 0.0001
Epoch: 004/005 | Batch 0400/2546 | Loss: 0.0056| Learning rate: 0.0001
Epoch: 004/005 | Batch 0500/2546 | Loss: 0.0015| Learning rate: 0.0001
Epoch: 004/005 | Batch 0600/2546 | Loss: 0.0021| Learning rate: 0.0001
Epoch: 004/005 | Batch 0700/2546 | Loss: 0.0072| Learning rate: 0.0001
Epoch: 004/005 | Batch 0800/2546 | Loss: 0.0046| Learning rate: 0.0001
Epoch: 004/005 | Batch 0900/2546 | Loss: 0.0014| Learning rate: 0.0001
Epoch: 004/005 | Batch 1000/2546 | Loss: 0.0042| Learning rate: 0.0001
Epoch: 004/005 | Batch 1100/2546 | Loss: 0.0043| Learning rate: 0.0001
Epoch: 004/005 | Batch 1200/2546 | Loss: 0.0022| Learning rate: 0.0001
Epoch: 004/005 | Batch 1300/2546 | Loss: 0.0010| Learning rate: 0.0001
Epoch:

In [ ]:
if scheduler == None:
    scheduler = StepLR(optimizer, step_size = 1, gamma = 1)
minibatch_loss, train_acc, valid_acc = train(model = model,
            num_epochs = 8,
            last_epoch = 5,
            train_loader = train_loader,
            validation_loader = validation_loader,
            test_loader = test_loader,
            optimizer = optimizer,
            device = DEVICE,
            scheduler = scheduler,
            checkpoint_dict = checkpoint_dict,
            scaler = scaler)

Epoch: 006/008 | Batch 0000/2546 | Loss: 0.0018| Learning rate: 0.0001
Epoch: 006/008 | Batch 0100/2546 | Loss: 0.0037| Learning rate: 0.0001
Epoch: 006/008 | Batch 0200/2546 | Loss: 0.0009| Learning rate: 0.0001
Epoch: 006/008 | Batch 0300/2546 | Loss: 0.0005| Learning rate: 0.0001
Epoch: 006/008 | Batch 0400/2546 | Loss: 0.0009| Learning rate: 0.0001
Epoch: 006/008 | Batch 0500/2546 | Loss: 0.0013| Learning rate: 0.0001
Epoch: 006/008 | Batch 0600/2546 | Loss: 0.0016| Learning rate: 0.0001
Epoch: 006/008 | Batch 0700/2546 | Loss: 0.0008| Learning rate: 0.0001
Epoch: 006/008 | Batch 0800/2546 | Loss: 0.0010| Learning rate: 0.0001
Epoch: 006/008 | Batch 0900/2546 | Loss: 0.0019| Learning rate: 0.0001
Epoch: 006/008 | Batch 1000/2546 | Loss: 0.0012| Learning rate: 0.0001
Epoch: 006/008 | Batch 1100/2546 | Loss: 0.0011| Learning rate: 0.0001
Epoch: 006/008 | Batch 1200/2546 | Loss: 0.0012| Learning rate: 0.0001
Epoch: 006/008 | Batch 1300/2546 | Loss: 0.0080| Learning rate: 0.0001
Epoch: